<div class="alert alert-info" style="border-color: white; border-radius: 10px; background-color: white;"> 

# <font color = 'mediumaquamarine'> Проект по SQL

<div class="alert alert-info" style="border-color: thistle; border-radius: 10px; background-color: thistle;"> 
    <font color = 'black'> 

**Описание проекта**  

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. 

Компания N решила быть на волне и купила крупный сервис для чтения книг по подписке. Первая задача аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

<div class="alert alert-info" style="border-color: lavender; border-radius: 10px; background-color: lavender;"> 
<div style="border:white 5px; padding: 30px">
    <font color = 'black'> 

**Задачи**  
1. Посчитать, сколько книг вышло после 1 января 2000 года;  
2. Для каждой книги посчитать количество обзоров и среднюю оценку;  
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (так мы исключим из анализа брошюры);  
4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;  
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.  


<div class="alert alert-info" style="border-color: slategray; border-radius: 10px; background-color: white;"> 
<div style="border:white 5px; padding: 30px">
    <font color = 'black'> 
        
**Описание данных**  

**Таблица books**  
Содержит данные о книгах:  
- `book_id` — идентификатор книги;  
- `author_id` — идентификатор автора;  
- `title` — название книги;  
- `num_pages` — количество страниц;  
- `publication_date` — дата публикации книги;  
- `publisher_id` — идентификатор издателя.    

**Таблица authors**  
Содержит данные об авторах:  
- `author_id` — идентификатор автора;  
- `author` — имя автора.  

**Таблица publishers**   
Содержит данные об издательствах:  
- `publisher_id` — идентификатор издательства;  
- `publisher` — название издательства.  

**Таблица ratings**  
Содержит данные о пользовательских оценках книг:  
- `rating_id` — идентификатор оценки;  
- `book_id` — идентификатор книги;   
- `username` — имя пользователя, оставившего оценку;  
- `rating` — оценка книги.  

**Таблица reviews**  
Содержит данные о пользовательских обзорах на книги:  
- `review_id` — идентификатор обзора;  
- `book_id` — идентификатор книги;  
- `username` — имя пользователя, написавшего обзор;  
- `text` — текст обзора. 

In [1]:
# Загрузим библиотеки
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
!pip install psycopg2-binary # выходила ошибка, в интернете нашла такое решение

In [2]:
# Устанавливаем параметры
db_config = {
             'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db' # название базы данных
             }

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# Сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# Чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

На всякий случай: в Пачке и здесь в описании проекта были даны немного разные варианты подключения к базе данных, и так как я начала  делать этот проект еще во время проверки A/B-теста, тут я использую вариант из Пачки.

Подключение в базе данных есть, теперь изучим имеющиеся таблицы - выведем первые пять строк каждого датасета.

**Таблица books**

In [3]:
# Формируем запрос и выводим данные
query = '''SELECT * FROM books LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица authors**

In [4]:
# Формируем запрос и выводим данные
query = '''SELECT * FROM authors LIMIT 5'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица publishers** 

In [5]:
# Формируем запрос и выводим данные
query = '''SELECT * FROM publishers LIMIT 5'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица ratings**  

In [6]:
# Формируем запрос и выводим данные
query = '''SELECT * FROM ratings LIMIT 5'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица reviews** 

In [7]:
# Формируем запрос и выводим данные
query = '''SELECT * FROM reviews LIMIT 5'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Взглянув на предоставленные данные, можно сделать вывод, что у нас есть достаточно информации о каждой книге, ее авторе, размерах, рейтинге и даже отзывов читателей. Сами по себе таблицы соединены определенными колонками, например, по book_id.

Теперь можем приступить к созданию запросов.

## Cколько книг вышло после 1 января 2000 года

In [8]:
# Формируем запрос и выводим данные
books2000 = '''SELECT COUNT(*) as number_of_books_after_2000
               FROM books
               WHERE publication_date > '2000-01-01'
               '''
get_sql_data(books2000)

,number_of_books_after_2000
0,819


In [9]:
# Формируем запрос и выводим данные (включим 1 января в фильтрацию)
books2000 = '''SELECT COUNT(*) as number_of_books_after_2000
               FROM books
               WHERE publication_date >= '2000-01-01'
               '''
get_sql_data(books2000)

,number_of_books_after_2000
0,821


После 1 января 2000 года вышло 819 книг. Но если включить 1 января в запрос, то количество возрастет на 3 книги. 

## Количество обзоров и средняя оценка каждой книги

In [12]:
# Формируем запрос и выводим данные 
avg_and_cnt = '''SELECT b.book_id, 
                        b.title, 
                        COUNT(DISTINCT r.text) AS reviews_cnt, 
                        AVG(rt.rating) AS rating_avg
                 FROM books b
                 LEFT JOIN reviews r ON b.book_id = r.book_id
                 LEFT JOIN ratings rt ON b.book_id = rt.book_id
                 GROUP BY b.book_id
                 ORDER BY rating_avg DESC
               '''
get_sql_data(avg_and_cnt)

,book_id,title,reviews_cnt,rating_avg
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


In [13]:
get_sql_data(avg_and_cnt)['reviews_cnt'].sum()

2793

Для каждой книги мы определили количество обзоров, которые написали на них пользователи, а также средний рейтинг. В будущем было бы интересно добавить жанр книг в анализ, чтобы узнать, что люди оценивают лучше и что им нравится больше из жанров.

## Издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [14]:
# Формируем запрос и выводим данные 
best_publisher = '''SELECT p.publisher,
                           COUNT(b.book_id) AS books_count
                    FROM publishers p
                    LEFT JOIN books b ON p.publisher_id = b.publisher_id
                    WHERE b.num_pages > 50
                    GROUP BY p.publisher
                    ORDER BY books_count DESC
                    LIMIT 1;
                 '''
get_sql_data(best_publisher)

,publisher,books_count
0,Penguin Books,42


Издательством с наибольшим числом книг толще 50 страниц стало **Penguin Books**, выпустившее 42 книги.

## Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками)

In [15]:
# Формируем запрос и выводим данные 
best_author = '''WITH
                 cnt_50 AS (SELECT book_id
                            FROM ratings
                            GROUP BY book_id
                            HAVING COUNT(rating_id) >= 50)
                 SELECT a.author,
                        AVG(r.rating) AS avg_rating
                 FROM books as b
                 INNER JOIN authors a ON b.author_id = a.author_id
                 INNER JOIN cnt_50 ON b.book_id = cnt_50.book_id
                 INNER JOIN ratings r ON cnt_50.book_id = r.book_id
                 GROUP BY a.author  
                 ORDER BY avg_rating DESC
                 LIMIT 1;
               '''
get_sql_data(best_author)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автором с самой высокой средней оценкой у книг, у которых в целом по количеству было от 50 оценок, стала **J.K. Rowling** (Джоа́н Ро́улинг - наиболее известная благодаря своей серии книг по Гарри Поттеру) в союзе с **Mary GrandPré** (Мэри ГрандПре - американский иллюстратор, наиболее известная своими иллюстрациями обложек и глав книг о Гарри Поттере). Их средняя оценка равна примерно 4,29.

## Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [16]:
# Формируем запрос и выводим данные 
avg_rewiew = '''SELECT AVG(review_count) AS average_review_count
                FROM (SELECT COUNT(review_id) AS review_count
                      FROM reviews
                      WHERE username IN (
                                        SELECT username
                                        FROM ratings
                                        GROUP BY username
                                        HAVING COUNT(rating_id) > 48
                                        )
                      GROUP BY username
               ) AS cnt_rewiew_48
            '''
get_sql_data(avg_rewiew)

,average_review_count
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно 24 обзорам. Это может указывать на то, что пользователи, которые активно оценивают книги, также склонны оставлять обзоры. Но оставлять обзоры такие пользователи предпочитают в среднем в два раза меньше.

<div class="alert alert-info" style="border-color: lavender; border-radius: 10px; background-color: lavender;"> 
<div style="border:white 5px; padding: 30px">
    <font color = 'black'> 

Соберем результаты нашего анализа в единый вывод:  
- После 1 января 2000 года вышло **819** книг;  
- Были определены количество обзоров и средний рейтинг каждой книги: самый высокий средний рейтинг составил **5** с **двумя** обзорами, а самый низкий средний рейтинг - **1,5** с таким же количеством обзоров;  
- Издательством с наибольшим числом книг толще 50 страниц стало **Penguin Books**, выпустившее 42 книги;  
- Автором с самой высокой средней оценкой у книг, у которых в целом по количеству было от 50 оценок, стала **J.K. Rowling** в союзе с иллюстратором **Mary GrandPré** - известные по серии книг о Гарри Поттере. Их средняя оценка равна **4,29**;  
-  Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно **24 обзорам**. Это может указывать на то, что пользователи, которые активно оценивают книги, также склонны оставлять обзоры.  

В качестве рекомендации можно предложить стимулировать пользователей к оставлению обзоров, например, предоставлять бонусы или вознаграждения за оставленные обзоры. Это поможет создать активное сообщество пользователей и повысить вовлеченность. Также в будущем было бы интересно посмотреть подробнее на книги, которые получили наиболее высокие рейтинги и количество отзывов - по этим данным мы бы смогли узнать, какие книги привлекают пользователей больше всего и после этого, по возможности, расширить книжные рекомендации похожими книгами. 